# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
import re
from collections import Counter
from collections import defaultdict
import random
import nltk
from nltk import bigrams, trigrams
nltk.download('gutenberg')

from nltk.corpus import gutenberg


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/leiluk1/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


### Norvig's solution: 

In [2]:
def words(text): 
    return re.findall(r'\w+', text.lower())

CORPUS_FILE = 'data/big.txt'
words_dict = Counter(words(open(CORPUS_FILE).read()))
N = sum(words_dict.values())

def P(word): 
    "Probability of `word`."
    return words_dict[word] / N

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in words_dict)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def norvig_correction(sentence):
    "Most probable spelling correction for words in input sentence."
    corrected = []
    for word in sentence: 
        corrected.append(correction(word))
    return corrected

In [3]:
# Test norvig's approach on example "Something went wrong"
norvig_correction('Smthing wrnt wong'.split())

['nothing', 'went', 'long']

### My solution based on N-grams:


In [4]:
# Read bigrams with their corresponding frequencies from files
BIGRAMS_FILE_1 = 'data/coca_all_links.txt'
BIGRAMS_FILE_2 = 'data/bigrams.txt'

bigram_model = defaultdict(lambda: defaultdict(int))

with open(BIGRAMS_FILE_1, 'r', encoding='latin1') as file:
    next(file)  # Skip the first line
    for line in file:
        freq, w1, w2, _, _ = line.strip().split()
        bigram_model[w1][w2] = int(freq)

with open(BIGRAMS_FILE_2, 'r', encoding='latin1') as file:
    for line in file:
        freq, w1, w2 = line.strip().split()
        bigram_model[w1][w2] = int(freq)

# Also get bigrams from the corpus file 'big.txt' using nltk bigrams
for sentence in nltk.sent_tokenize(open(CORPUS_FILE).read()):
    for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
        bigram_model[w1][w2] += 1


# Transform the frequencies into probabilities
for w1 in bigram_model:
    total_freq = float(sum(bigram_model[w1].values()))
    for w2 in bigram_model[w1]:
        bigram_model[w1][w2] /= total_freq



next(iter(bigram_model.items()))

('abandoned',
 defaultdict(int,
             {'building': 0.014885382554331646,
              'buildings': 0.023816612086930634,
              'car': 0.007889252753795772,
              'cars': 0.010866329264662102,
              'child': 0.004316760940756177,
              'children': 0.01220601369455195,
              'homes': 0.004019053289669544,
              'house': 0.011759452217922,
              'houses': 0.008931229532598988,
              'mine': 0.010270913962488836,
              'mines': 0.008038106579339089,
              'railroad': 0.00506103006847276,
              'a': 0.01726704376302471,
              'after': 0.01012206013694552,
              'all': 0.008186960404882405,
              'and': 0.04257219410538851,
              'any': 0.007293837451622507,
              'as': 0.016522774635308126,
              'at': 0.012057159869008634,
              'babies': 0.004019053289669544,
              'because': 0.009377791009228937,
              'by': 0.101071747543

In [5]:
# Get trigrams with their corresponding frequencies from file with fivegrams
FIVEGRAMS_FILE = 'data/fivegrams.txt'

trigram_model = defaultdict(lambda: defaultdict(int))

with open(FIVEGRAMS_FILE, 'r', encoding='latin1') as file:
    for line in file:
        freq, w1, w2, w3, w4, w5 = line.strip().split()
        if (w1, w2) not in trigram_model.items():
            trigram_model[(w1, w2)][w3] = int(freq)
        else:
             trigram_model[(w1, w2)][w3] += int(freq)
        
        if (w2, w3) not in trigram_model.items():
            trigram_model[(w2, w3)][w4] = int(freq)
        else:
             trigram_model[(w2, w3)][w4] += int(freq)
             
        if (w3, w4) not in trigram_model.items():
            trigram_model[(w3, w4)][w5] = int(freq)
        else:
             trigram_model[(w3, w4)][w5] += int(freq)


# Also get trigrams from the corpus file 'big.txt' using nltk trigrams
for sentence in  nltk.sent_tokenize(open(CORPUS_FILE).read()):
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        trigram_model[(w1, w2)][w3] += 1


    
# Transform the frequencies into probabilities
for w1w2 in trigram_model:
    total_freq = float(sum(trigram_model[w1w2].values()))
    for w3 in trigram_model[w1w2]:
        trigram_model[w1w2][w3] /= total_freq

next(iter(trigram_model.items()))

(('a', 'babe'), defaultdict(int, {'in': 1.0}))

In [6]:
def bigram_correction(sentence, N=2):
    "Most probable spelling correction for sentence using bigram model."
    sentence = [word.lower() for word in sentence]
    corrected_sentence = [correction(sentence[0])]

    for i in range(len(sentence) - N + 1):
        max_P = 0
        best_candidate = sentence[i + N - 1]

        for candidate in list(candidates(sentence[i + N - 1])):
            if bigram_model[sentence[i]][candidate] >= max_P:
                max_P = bigram_model[sentence[i]][candidate]
                best_candidate = candidate
    
        corrected_sentence.append(best_candidate)

    return corrected_sentence

In [7]:
bigram_correction('Smthing wrnt wong'.split())

['something', 'writ', 'gong']

In [8]:
def trigram_correction(sentence, N=3):
    "Most probable spelling correction for sentence using trigram model."
    sentence = [word.lower() for word in sentence]
    corrected_sentence = []
    for word in bigram_correction(sentence[:N - 1]):
        corrected_sentence.append(word)

    for i in range(len(sentence) - N + 1):
        max_P = 0
        best_candidate = sentence[i + N - 1]

        for candidate in list(candidates(sentence[i + 2])):
            if trigram_model[(sentence[i], sentence[i + 1])][candidate] >= max_P:
                max_P = trigram_model[(sentence[i], sentence[i + 1])][candidate]
                best_candidate = candidate
        
        corrected_sentence.append(best_candidate)

    return corrected_sentence

In [9]:
trigram_correction('Smthing wrnt wong'.split())

['something', 'writ', 'gong']

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

To implement a context-sensitive spelling corrector, I decided to use N-gram language models, specifically bigrams and trigrams models that take into account the near context in the sentence. For the N-gram datasets, I used the data provided in Moodle. For the bigram model, I used the files `bigrams.txt` and `coca_all_links.txt`, and for trigrams `fivegrams.txt`, from which I  retrieved the trigrams. Furthermore, I created both bigrams and trigrams using `nltk` from the `big.txt` corpus file. All used data can be found in the `data` folder.

After reading the data, I created the bigram and trigram models. These models contain probabilities for each bigram and trigram based on the datasets used. In terms of the spelling correction, I implemented the `bigram_correction` function for the bigram model and the `trigram_correction` function for the trigram model. The `bigram_correction` function iterates over the given sentence and computes the best candidates for each word using the bigram model. The `trigram_correction` function applies the `bigram_correction` function to the first two words of the sentence and then iterates over the rest of the sentence, computing the best candidates using the trigrams models. 

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [10]:
def corrupt_sentence(sentence, prob=0.5, mistake_type_prob=0.9):
    "Corrupt sentences from the test set by generating errors in words randomly."
    sent_tokens = re.findall(r'\w+', sentence)
    curr_idx = 0
    for token in sent_tokens:
        correction = token
        if random.random() < prob and not token.isnumeric():
            if random.random() < mistake_type_prob:
                correction = random.choice(list(edits1(token)))
            else: 
                correction = random.choice(list(edits2(token)))
            if correction == '':
                correction = token
        if token[0].isupper():
            correction = correction[0].upper() + correction[1:]
        idx = sentence.find(token, curr_idx)
        sentence = sentence[:idx] + correction + sentence[idx + len(token):]
        curr_idx = idx + len(correction)
    return sentence

In [11]:
# Use first 1500 meaningful sentences from Emma by Jane Austen from the Gutenberg electronic text archive (NLTK)
# https://www.nltk.org/book/ch02.html

emma_sentences = gutenberg.sents('austen-emma.txt')[3:1503]
correct_test_set, corrupted_test_set = [], []

# Create correct and generated corrupted test sets 
for sentence in emma_sentences[1:]:
    sentence = " ".join(sentence)
    correct_test_set.append(sentence)
    corrupted_test_set.append(corrupt_sentence(sentence))
  
for correct_words, corrupted in zip(correct_test_set[:5], corrupted_test_set[:5]):
    print(correct_words)
    print(corrupted)
    print()


She was the youngest of the two daughters of a most affectionate , indulgent father ; and had , in consequence of her sister ' s marriage , been mistress of his house from a very early period .
TShe was the youngtest on tyhe two xaughters of ea most affectionate , indulgent fakher ; and jhad , in consequezce of gher sister ' s mawrriage , veen misstress lf uis house mrom a very esrly permiod .

Her mother had died too long ago for her to have more than an indistinct remembrance of her caresses ; and her place had been supplied by an excellent woman as governess , who had fallen little short of a mother in affection .
Her mothecr had aied too long ajgoi for heyr to haie mordpe thal any gndisxinct rimembrance oaf her cargesses ; and her lace had been supplged by ib vexcellent woman gs governqess , bho haad fallee little shorx of a mothkr zin afpfection .

Sixteen years had Miss Taylor been in Mr . Woodhouse ' s family , less as a governess than a friend , very fond of both daughters , bu

In [12]:
def eval_result(correct_sentence, result):
    "Evaluate resulted sentence comparing each word with corresponding word in correct sentence."
    correct_words = 0
    for i in range(len(correct_sentence)):
        if correct_sentence[i].lower() == result[i].lower():
            correct_words += 1
    
    return correct_words


def eval_approaches(correct_input, corrupted_input):
    "Evaluate approaches and compute accuracy for each."
    acc_Norvig, acc_Bigram, acc_Trigram = 0, 0, 0
    total_words = 0

    for correct, corrupted in zip(correct_input, corrupted_input):
        result_Norvig = norvig_correction(corrupted.split())
        result_Bigram = bigram_correction(corrupted.split())
        result_Trigram = trigram_correction(corrupted.split())
        
        acc_Norvig += eval_result(correct.split(), result_Norvig)
        acc_Bigram += eval_result(correct.split(), result_Bigram)
        acc_Trigram += eval_result(correct.split(), result_Trigram)

        total_words += len(correct.split())

    acc_Norvig /= total_words
    acc_Bigram /= total_words
    acc_Trigram /= total_words
    print(f"Norvig's spelling corrector accuracy:\t{acc_Norvig:.3f}\nContext Bigram corrector accuracy:\t{acc_Bigram:.3f}\nContext Trigram corrector accuracy:\t{acc_Trigram:.3f}")

In [13]:
# Compare approaches by accuracy
eval_approaches(correct_test_set, corrupted_test_set)

Norvig's spelling corrector accuracy:	0.703
Context Bigram corrector accuracy:	0.669
Context Trigram corrector accuracy:	0.627


However, I did not improve the accuracy of Norvig's approach. Maybe more strong or complex solutions, bigger N or not necessarily N-gram approach, might be considered.